In [2]:
import polars as pl

In [3]:
df = pl.read_parquet("Assets/flights.parquet")
df

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
i16,i8,i8,i16,i16,i16,i16,i16,i16,str,i16,str,str,str,i16,i16,i8,i8,datetime[ns]
2013,1,1,517,515,2,830,819,11,"""UA""",1545,"""N14228""","""EWR""","""IAH""",227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,"""UA""",1714,"""N24211""","""LGA""","""IAH""",227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,"""AA""",1141,"""N619AA""","""JFK""","""MIA""",160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,"""B6""",725,"""N804JB""","""JFK""","""BQN""",183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,"""DL""",461,"""N668DN""","""LGA""","""ATL""",116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,"""UA""",1696,"""N39463""","""EWR""","""ORD""",150,719,5,58,2013-01-01 05:00:00
2013,1,1,555,600,-5,913,854,19,"""B6""",507,"""N516JB""","""EWR""","""FLL""",158,1065,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,709,723,-14,"""EV""",5708,"""N829AS""","""LGA""","""IAD""",53,229,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,838,846,-8,"""B6""",79,"""N593JB""","""JFK""","""MCO""",140,944,6,0,2013-01-01 06:00:00


## Conditions:

 -> Using Origin, Destination, Carrier, Flight and time_hour fields

- Find flights with an arrival time less than 1000 and that is not null.
- Flights made between june and august
- Flights whose origin is JFK

### Vectorized Query

In [10]:
%%timeit
df.lazy().filter(
    (pl.col("arr_time") < 1000) & (pl.col("arr_time").is_not_null())
    & (pl.col("month").is_between(6,8)) & (pl.col("origin") == "JFK")
)

19.8 µs ± 2.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
df.lazy().filter(
    (pl.col("arr_time") < 1000) & (pl.col("arr_time").is_not_null())
    & (pl.col("month").is_between(6,8)) & (pl.col("origin") == "JFK")
).select(pl.col(["origin","dest","carrier","flight","time_hour"])).collect()

origin,dest,carrier,flight,time_hour
str,str,str,i16,datetime[ns]
"""JFK""","""PSE""","""B6""",739,2013-06-01 23:00:00
"""JFK""","""BQN""","""B6""",725,2013-06-01 05:00:00
"""JFK""","""MIA""","""AA""",701,2013-06-01 05:00:00
"""JFK""","""IAD""","""EV""",5716,2013-06-01 06:00:00
"""JFK""","""LAX""","""UA""",1159,2013-06-01 06:00:00
"""JFK""","""SJU""","""B6""",715,2013-06-01 06:00:00
"""JFK""","""SFO""","""UA""",303,2013-06-01 06:00:00
"""JFK""","""FLL""","""B6""",315,2013-06-01 06:00:00
"""JFK""","""CLT""","""US""",1125,2013-06-01 06:00:00
